import libraries

In [1]:
import pandas as pd
import numpy as np
from Levenshtein import distance as lev_distance
from nltk.tokenize import word_tokenize

In [2]:
# read nid data original and predicted
df_original_nid=pd.read_csv("national_id/high_resolution_final_ids.csv")
df_predicted_nid=pd.read_csv("national_id/high_resolution_National_IDs_predicted.csv")


# read credit card data original and predicted
df_original_credit=pd.read_csv("credit_card/cards200V2.csv")
df_predicted_credit=pd.read_csv("credit_card/ocr_result_test.csv")

In [3]:
#standarize column names in original and predicted
df_predicted_nid.rename(columns={"FN":"First Name","LN":"Last Name","Add1":"Address","Add2":"District-Govrnate","Id":"National ID"},inplace=True)

In [4]:
df_predicted_nid

,First Name,Last Name,Address,District-Govrnate,National ID,image_path
0,-ايمن,أحمذ محمد سيد,ع 586 نثر مصر القرئقل,التجمع الاول. - القاهرة.,238610962102906,/content/drive/MyDrive/NID_Data/image1.jpg
1,محمد,عبد المئعم محمد السيد,عزبة المهاجرين,المنتزه .- الاسكندرية,27010200202071,/content/drive/MyDrive/NID_Data/image2.jpg
2,عيد,سعيد سلامة الحمادين,حى الحمادين,الشيغ زويد - شمال سيناء,24308233400071,/content/drive/MyDrive/NID_Data/image3.jpg
3,احمد,خالد محمد على,كلا حين ابنود,مركز قنا - قنا,29010102701652,/content/drive/MyDrive/NID_Data/image4.jpg
4,فاطمه,NaN,"""ش مسجدالرحمه-فيكتوريا",اول المنتزه .- الاسكندرية,29303210200586,/content/drive/MyDrive/NID_Data/image5.jpg
5,محمد,نهاد محمد صديق الشوربجى,عمارة /ا١‏ مساكن الصباط,اول السلام - القاهره,29602060101417,/content/drive/MyDrive/NID_Data/image6.jpg
6,علياء,محمؤود احمد نصر شلبى,6 عبدالوهاب زيدان-ميدان فيكتوريا,الساحل - القاهره,29701050101841,/content/drive/MyDrive/NID_Data/image7.jpg
7,احمد,حسين عواض حسن شاهين,نهاية وابور الفولى من بن واصل,ثان الرمل - الاسكندرية,29204180200995,/content/drive/MyDrive/NID_Data/image8.jpg
8,محمود,عصام عبدالعزيز قطب محمد,عبدالقادر - الحكماء,الزقازيق ثان - الشرقية,28909091300595,/content/drive/MyDrive/NID_Data/image9.jpg
9,احمد,منير شوكت احمد,NaN,مركز بنها - القليوبية,29006141400948,/content/drive/MyDrive/NID_Data/image10.jpg


In [5]:
df_original_nid

,Image,First Name,Last Name,Address,District-Govrnate,National ID
0,Image1,ايمن,احمد محمد سيد,ع ٣٨٤ دار مصر القرنفل,التجمع الاول-القاهره,28610162102956
1,image2,محمد,عبدالمنعم محمد السيد,عزبه المهاجرين,المنتزه- الاسكندريه,27010200202071
2,image3,عيد,سعيد سلامه الحمادين,حي الحمادين,الشيخ زويد - شمال سيناء,24308233400071
3,image4,احمد,خالد محمد على,كلا حين ابنود,مركز قنا- قنا,29010102701652
4,image5,فاطمه,خليل حسن خليل محمد,٢ ش مسجد الرحمه-فيكتوريا,اول المنتزه - الاسكندرية,29203210200586
5,image6,محمد,نهاد محمد صدقي الشوربجي,عمارة ١٧ مساكن الضباط,اول السلام - القاهره,29202060101417
6,image7,علياء,محمود احمد نصر شلبي,٢٨ عبدالوهاب زيدان-ميدان فيكتوريا,الساحل- القاهره,29701050101841
7,Image8,احمد,حسين عواض حسن شاهين,نهاية وابور الفولى من بن واصل,ثان الرمل - الاسكندرية,29204180200995
8,image9,محمود,عصام عبد العزيز قطب محمد,٩ ش عبد القادر - الحكماء,الزقازيق ثان الشرقية,28909091300595
9,image10,احمد,منير شوكت احمد,بطا,مركز بنها - القليوبية,29006141400948


In [6]:
# standarize path column to be primary key for both dfs 
df_original_nid['path'] = df_original_nid['Image'].apply(lambda x: f'{x}.jpg'.lower())
df_predicted_nid['path'] = df_predicted_nid['image_path'].apply(lambda x: x.split('/')[-1].lower())
df_predicted_nid.drop(columns="image_path",inplace=True)
df_original_nid.drop(columns="Image",inplace=True)


In [7]:
# make sure data are on the same order by inner merge 
inner_df=pd.merge(df_original_nid,df_predicted_nid, on="path", how="inner")
print(f"Evaluation based on {len(inner_df)} images")
df_original_nid_1 = inner_df[[col for col in inner_df.columns if '_x' in col] ].rename(columns=lambda x: x.replace('_x', ''))
df_predicted_nid = inner_df[[col for col in inner_df.columns if '_y' in col] ].rename(columns=lambda x: x.replace('_y', ''))

Evaluation based on 10 images


Evaluation Metrics Implementation

In [8]:

def levenshtein_distance(s1, s2):
    return lev_distance(str(s1), str(s2))

def word_error_rate(reference, hypothesis):
    ref_words = word_tokenize(str(reference))
    hyp_words = word_tokenize(str(hypothesis))
    distance = lev_distance(ref_words, hyp_words)
    return distance / len(ref_words)

def character_error_rate(reference, hypothesis):
    distance = lev_distance(str(reference), str(hypothesis))
    return distance / len(str(reference))


def precision_recall_f1(reference, hypothesis):
    ref_words = set(word_tokenize(str(reference)))
    hyp_words = set(word_tokenize(str(hypothesis)))
    
    true_positives = len(ref_words.intersection(hyp_words))
    false_positives = len(hyp_words - ref_words)
    false_negatives = len(ref_words - hyp_words)
    
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1

def evaluate_ocr_results(extracted_df, ground_truth_df):
    if not extracted_df.columns.equals(ground_truth_df.columns):
        raise ValueError("Columns in extracted_df and ground_truth_df do not match")
    
    results = {}
    for column in extracted_df.columns:
        column_results = {
            'levenshtein': [],
            'wer': [],
            'cer': [],
            'precision': [],
            'recall': [],
            'f1': []
        }
        
        for ext, truth in zip(extracted_df[column], ground_truth_df[column]):
            column_results['levenshtein'].append(levenshtein_distance(ext, truth))
            column_results['wer'].append(word_error_rate(truth, ext))
            column_results['cer'].append(character_error_rate(truth, ext))
            p, r, f1 = precision_recall_f1(truth, ext)
            column_results['precision'].append(p)
            column_results['recall'].append(r)
            column_results['f1'].append(f1)
        
        results[column] = {metric: np.mean(values) for metric, values in column_results.items()}
    
    return results

def print_results(results):
    print("Evaluation Results:")
    for field, metrics in results.items():
        print(f"\n{field}:")
        print(f"  Avg Levenshtein Distance: {metrics['levenshtein']:.2f}")
        print(f"  Word Error Rate: {metrics['wer']:.2%}")
        print(f"  Character Error Rate: {metrics['cer']:.2%}")
        print(f"  Precision: {metrics['precision']:.2%}")
        print(f"  Recall: {metrics['recall']:.2%}")
        print(f"  F1 Score: {metrics['f1']:.2%}")


In [9]:
results = evaluate_ocr_results(df_predicted_nid, df_original_nid_1)
print_results(results)

Evaluation Results:

First Name:
  Avg Levenshtein Distance: 0.10
  Word Error Rate: 10.00%
  Character Error Rate: 2.50%
  Precision: 90.00%
  Recall: 90.00%
  F1 Score: 90.00%

Last Name:
  Avg Levenshtein Distance: 2.90
  Word Error Rate: 37.33%
  Character Error Rate: 15.84%
  Precision: 65.83%
  Recall: 66.00%
  F1 Score: 65.71%

Address:
  Avg Levenshtein Distance: 2.60
  Word Error Rate: 47.67%
  Character Error Rate: 20.80%
  Precision: 56.50%
  Recall: 52.33%
  F1 Score: 53.80%

District-Govrnate:
  Avg Levenshtein Distance: 1.80
  Word Error Rate: 64.50%
  Character Error Rate: 9.18%
  Precision: 65.00%
  Recall: 72.17%
  F1 Score: 67.60%

National ID:
  Avg Levenshtein Distance: 0.50
  Word Error Rate: 30.00%
  Character Error Rate: 3.57%
  Precision: 70.00%
  Recall: 70.00%
  F1 Score: 70.00%


 Analysis for Arabic character position error

In [10]:
# drop national ids as its just numbers we would work on alpahpitical only
df_original_nid_1.drop(columns="National ID",inplace= True)
df_predicted_nid.drop(columns="National ID",inplace= True)

In [11]:
from collections import defaultdict

def get_char_form(char, position):
    """
    Determine the Unicode form of an Arabic character based on its position.
    position can be: 'isolated', 'initial', 'medial', or 'final'
    """
    # Dictionary mapping base characters to their positional forms
    # Format: 'base': (isolated, initial, medial, final)
    char_forms = {
        'ب': ('\u0628', '\uFE91', '\uFE92', '\uFE90'),  # beh
        'ت': ('\u062A', '\uFE97', '\uFE98', '\uFE96'),  # teh
        'ث': ('\u062B', '\uFE9B', '\uFE9C', '\uFE9A'),  # theh
        'ج': ('\u062C', '\uFE9F', '\uFEA0', '\uFE9E'),  # jeem
        'ح': ('\u062D', '\uFEA3', '\uFEA4', '\uFEA2'),  # hah
        'خ': ('\u062E', '\uFEA7', '\uFEA8', '\uFEA6'),  # khah
        'س': ('\u0633', '\uFEB3', '\uFEB4', '\uFEB2'),  # seen
        'ش': ('\u0634', '\uFEB7', '\uFEB8', '\uFEB6'),  # sheen
        'ص': ('\u0635', '\uFEBB', '\uFEBC', '\uFEBA'),  # sad
        'ض': ('\u0636', '\uFEBF', '\uFEC0', '\uFEBE'),  # dad
        'ط': ('\u0637', '\uFEC3', '\uFEC4', '\uFEC2'),  # tah
        'ظ': ('\u0638', '\uFEC7', '\uFEC8', '\uFEC6'),  # zah
        'ع': ('\u0639', '\uFECB', '\uFECC', '\uFECA'),  # ain
        'غ': ('\u063A', '\uFECF', '\uFED0', '\uFECE'),  # ghain
        'ف': ('\u0641', '\uFED3', '\uFED4', '\uFED2'),  # feh
        'ق': ('\u0642', '\uFED7', '\uFED8', '\uFED6'),  # qaf
        'ك': ('\u0643', '\uFEDB', '\uFEDC', '\uFEDA'),  # kaf
        'ل': ('\u0644', '\uFEDF', '\uFEE0', '\uFEDE'),  # lam
        'م': ('\u0645', '\uFEE3', '\uFEE4', '\uFEE2'),  # meem
        'ن': ('\u0646', '\uFEE7', '\uFEE8', '\uFEE6'),  # noon
        'ه': ('\u0647', '\uFEEB', '\uFEEC', '\uFEEA'),  # heh
        'ي': ('\u064A', '\uFEF3', '\uFEF4', '\uFEF2'),  # yeh
    }
    
    positions = {'isolated': 0, 'initial': 1, 'medial': 2, 'final': 3}
    
    if char in char_forms and position in positions:
        return char_forms[char][positions[position]]
    return char

def analyze_word(word):
    """
    Analyze an Arabic word and determine the correct form for each character.
    Returns a list of dictionaries containing character analysis.
    """
    results = []
    chars = list(word)
    
    for i, char in enumerate(chars):
        # Determine position
        if len(chars) == 1:
            position = 'isolated'
        elif i == 0:
            position = 'initial'
        elif i == len(chars) - 1:
            position = 'final'
        else:
            position = 'medial'
            
        correct_form = get_char_form(char, position)
        
        analysis = {
            'original': char,
            'position': position,
            'correct_form': correct_form,
            'unicode_value': f'U+{ord(correct_form):04X}',
            'is_connecting': char in 'بتثجحخسشصضطظعغفقكلمنهي'
        }
        results.append(analysis)
    
    return results

   

class ArabicCharacterAnalyzer:
    def __init__(self):
        self.reset_metrics()
        
    def reset_metrics(self):
        """Initialize metrics tracking dictionaries"""
        # Structure: char -> position -> metrics
        self.char_position_metrics = defaultdict(lambda: {
            'initial': {'correct': 0, 'total': 0},
            'medial': {'correct': 0, 'total': 0},
            'final': {'correct': 0, 'total': 0},
            'isolated': {'correct': 0, 'total': 0},
            'overall': {'correct': 0, 'total': 0}
        })
        
        # Structure: position -> metrics
        self.position_metrics = {
            'initial': {'correct': 0, 'total': 0},
            'medial': {'correct': 0, 'total': 0},
            'final': {'correct': 0, 'total': 0},
            'isolated': {'correct': 0, 'total': 0}
        }

    def analyze_position(self, char, index, word_length):
        """Determine character position in word"""
        if word_length == 1:
            return 'isolated'
        elif index == 0:
            return 'initial'
        elif index == word_length - 1:
            return 'final'
        else:
            return 'medial'

    def evaluate_dataframes(self, gt_df, ocr_df):
        """Evaluate ground truth against OCR output"""
        self.reset_metrics()
        
        # Process each column
        for col in gt_df.columns:
            for gt_sentence, ocr_sentence in zip(gt_df[col], ocr_df[col]):
                if pd.isna(gt_sentence) or pd.isna(ocr_sentence):
                    continue
                    
                self.compare_sentences(str(gt_sentence), str(ocr_sentence))
        
        return self.generate_report()

    def compare_sentences(self, gt_sentence, ocr_sentence):
        """Compare ground truth and OCR output sentences"""
        gt_words = gt_sentence.split()
        ocr_words = ocr_sentence.split()
        
        for gt_word, ocr_word in zip(gt_words, ocr_words):
            # Compare characters
            for i, gt_char in enumerate(gt_word):
                position = self.analyze_position(gt_char, i, len(gt_word))
                
                # Get corresponding OCR character
                ocr_char = ocr_word[i] if i < len(ocr_word) else None
                
                # Update metrics
                self.char_position_metrics[gt_char][position]['total'] += 1
                self.char_position_metrics[gt_char]['overall']['total'] += 1
                self.position_metrics[position]['total'] += 1
                
                if ocr_char and gt_char == ocr_char:
                    self.char_position_metrics[gt_char][position]['correct'] += 1
                    self.char_position_metrics[gt_char]['overall']['correct'] += 1
                    self.position_metrics[position]['correct'] += 1

    def generate_report(self):
        """Generate comprehensive evaluation report"""
        # Create character position analysis table
        char_data = []
        
        for char, metrics in self.char_position_metrics.items():
            row = {
                'Character': char,
                'Initial_Accuracy': metrics['initial']['correct'] / metrics['initial']['total'] 
                    if metrics['initial']['total'] > 0 else 0,
                'Initial_Total': metrics['initial']['total'],
                'Medial_Accuracy': metrics['medial']['correct'] / metrics['medial']['total']
                    if metrics['medial']['total'] > 0 else 0,
                'Medial_Total': metrics['medial']['total'],
                'Final_Accuracy': metrics['final']['correct'] / metrics['final']['total']
                    if metrics['final']['total'] > 0 else 0,
                'Final_Total': metrics['final']['total'],
                'Isolated_Accuracy': metrics['isolated']['correct'] / metrics['isolated']['total']
                    if metrics['isolated']['total'] > 0 else 0,
                'Isolated_Total': metrics['isolated']['total'],
                'Overall_Accuracy': metrics['overall']['correct'] / metrics['overall']['total']
                    if metrics['overall']['total'] > 0 else 0,
                'Overall_Total': metrics['overall']['total']
            }
            char_data.append(row)
        
        # Create position summary row
        position_summary = {
            'Character': 'OVERALL',
            'Initial_Accuracy': self.position_metrics['initial']['correct'] / self.position_metrics['initial']['total']
                if self.position_metrics['initial']['total'] > 0 else 0,
            'Initial_Total': self.position_metrics['initial']['total'],
            'Medial_Accuracy': self.position_metrics['medial']['correct'] / self.position_metrics['medial']['total']
                if self.position_metrics['medial']['total'] > 0 else 0,
            'Medial_Total': self.position_metrics['medial']['total'],
            'Final_Accuracy': self.position_metrics['final']['correct'] / self.position_metrics['final']['total']
                if self.position_metrics['final']['total'] > 0 else 0,
            'Final_Total': self.position_metrics['final']['total'],
            'Isolated_Accuracy': self.position_metrics['isolated']['correct'] / self.position_metrics['isolated']['total']
                if self.position_metrics['isolated']['total'] > 0 else 0,
            'Isolated_Total': self.position_metrics['isolated']['total'],
            'Overall_Accuracy': sum(p['correct'] for p in self.position_metrics.values()) / 
                              sum(p['total'] for p in self.position_metrics.values()),
            'Overall_Total': sum(p['total'] for p in self.position_metrics.values())
        }
        
        # Create DataFrame and sort by overall accuracy
        df = pd.DataFrame(char_data)
        df = df.sort_values('Overall_Accuracy', ascending=False)
        
        # Add position summary row at the bottom
        df = pd.concat([df, pd.DataFrame([position_summary])], ignore_index=True)
        
        # Format accuracies as percentages
        for col in df.columns:
            if 'Accuracy' in col:
                df[col] = df[col].apply(lambda x: f"{x:.2%}")
                
        return df

def test_analyzer():
  
    
    # Initialize and run analyzer
    analyzer = ArabicCharacterAnalyzer()
    results = analyzer.evaluate_dataframes( df_original_nid_1,df_predicted_nid)
    
    # Display results
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    print(results)
    
    return results

if __name__ == "__main__":
    results = test_analyzer()



   Character Initial_Accuracy  Initial_Total Medial_Accuracy  Medial_Total  \
0          ث          100.00%              2           0.00%             0   
1          ى            0.00%              0           0.00%             0   
2          ت            0.00%              0         100.00%             4   
3          ج            0.00%              0         100.00%             3   
4          ص          100.00%              1         100.00%             4   
5          ء            0.00%              0           0.00%             0   
6          ح          100.00%              4          94.12%            17   
7          و          100.00%              2          93.33%            15   
8          م           92.86%             14          92.86%            28   
9          ك          100.00%              1          87.50%             8   
10         ب          100.00%              2          90.91%            11   
11         س          100.00%              4          77.78%    

In [12]:
results

,Character,Initial_Accuracy,Initial_Total,Medial_Accuracy,Medial_Total,Final_Accuracy,Final_Total,Isolated_Accuracy,Isolated_Total,Overall_Accuracy,Overall_Total
0,ث,100.00%,2,0.00%,0,0.00%,0,0.00%,0,100.00%,2
1,ى,0.00%,0,0.00%,0,100.00%,2,0.00%,0,100.00%,2
2,ت,0.00%,0,100.00%,4,100.00%,1,0.00%,0,100.00%,5
3,ج,0.00%,0,100.00%,3,0.00%,0,0.00%,0,100.00%,3
4,ص,100.00%,1,100.00%,4,0.00%,0,0.00%,0,100.00%,5
5,ء,0.00%,0,0.00%,0,100.00%,2,0.00%,0,100.00%,2
6,ح,100.00%,4,94.12%,17,0.00%,0,0.00%,0,95.24%,21
7,و,100.00%,2,93.33%,15,0.00%,0,0.00%,0,94.12%,17
8,م,92.86%,14,92.86%,28,66.67%,3,0.00%,0,91.11%,45
9,ك,100.00%,1,87.50%,8,0.00%,0,0.00%,0,88.89%,9


In [13]:
import pandas as pd
import numpy as np
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt

class MultiColumnArabicEvaluator:
    def __init__(self):
        self.reset_metrics()
    
    def reset_metrics(self):
        """Reset all metrics for new evaluation"""
        self.char_metrics = defaultdict(lambda: {
            'total': 0,
            'correct': 0,
            'position_errors': defaultdict(int),
            'substitution_errors': defaultdict(int)
        })
        self.position_metrics = defaultdict(lambda: {
            'total': 0,
            'correct': 0,
            'errors': defaultdict(int)
        })
        self.column_metrics = defaultdict(lambda: {
            'total_chars': 0,
            'correct_chars': 0,
            'char_accuracy': 0,
            'total_words': 0,
            'correct_words': 0,
            'word_accuracy': 0,
            'common_errors': defaultdict(int)
        })

    def analyze_position(self, char, index, word_length):
        """Determine character position in word"""
        if word_length == 1:
            return 'isolated'
        elif index == 0:
            return 'initial'
        elif index == word_length - 1:
            return 'final'
        else:
            return 'medial'

    def compare_sentences(self, gt_sentence, ocr_sentence):
        """Compare ground truth and OCR output sentences"""
        gt_words = gt_sentence.split()
        ocr_words = ocr_sentence.split()
        
        word_results = []
        for gt_word, ocr_word in zip(gt_words, ocr_words):
            char_comparisons = []
            
            # Compare characters
            for i, gt_char in enumerate(gt_word):
                gt_pos = self.analyze_position(gt_char, i, len(gt_word))
                
                # Find corresponding OCR character
                ocr_char = ocr_word[i] if i < len(ocr_word) else None
                ocr_pos = self.analyze_position(ocr_char, i, len(ocr_word)) if ocr_char else None
                
                comparison = {
                    'ground_truth': {
                        'char': gt_char,
                        'position': gt_pos,
                    },
                    'ocr_output': {
                        'char': ocr_char,
                        'position': ocr_pos,
                    },
                    'is_correct': gt_char == ocr_char,
                    'position_match': gt_pos == ocr_pos if ocr_char else False
                }
                char_comparisons.append(comparison)
                
                # Update metrics
                if gt_char:
                    self.char_metrics[gt_char]['total'] += 1
                    self.position_metrics[gt_pos]['total'] += 1
                    
                    if ocr_char:
                        if gt_char == ocr_char:
                            self.char_metrics[gt_char]['correct'] += 1
                            if gt_pos == ocr_pos:
                                self.position_metrics[gt_pos]['correct'] += 1
                            else:
                                self.position_metrics[gt_pos]['errors'][ocr_pos] += 1
                        else:
                            self.char_metrics[gt_char]['substitution_errors'][ocr_char] += 1
            
            word_results.append({
                'ground_truth': gt_word,
                'ocr_output': ocr_word,
                'char_comparisons': char_comparisons,
                'word_correct': gt_word == ocr_word
            })
            
        return word_results

    def evaluate_dataframes(self, gt_df, ocr_df):
        """
        Evaluate OCR output against ground truth across multiple columns
        
        Parameters:
        gt_df: DataFrame with ground truth sentences
        ocr_df: DataFrame with OCR output sentences
        """
        self.reset_metrics()
        column_results = {}
        
        # Verify both DataFrames have the same shape
        if gt_df.shape != ocr_df.shape:
            raise ValueError("Ground truth and OCR DataFrames must have the same shape")
        
        # Process each column
        for col in gt_df.columns:
            column_results[col] = []
            self.column_metrics[col] = {
                'total_chars': 0,
                'correct_chars': 0,
                'total_words': 0,
                'correct_words': 0,
                'error_types': defaultdict(int)
            }
            
            # Process each row in the column
            for gt_sentence, ocr_sentence in zip(gt_df[col], ocr_df[col]):
                if pd.isna(gt_sentence) or pd.isna(ocr_sentence):
                    continue
                    
                sentence_comparison = self.compare_sentences(str(gt_sentence), str(ocr_sentence))
                column_results[col].append({
                    'ground_truth': gt_sentence,
                    'ocr_output': ocr_sentence,
                    'word_comparisons': sentence_comparison
                })
                
                # Update column metrics
                self.update_column_metrics(col, sentence_comparison)
        
        return self.generate_report(column_results)

    def update_column_metrics(self, column, sentence_comparison):
        """Update metrics for a specific column"""
        for word_comp in sentence_comparison:
            self.column_metrics[column]['total_words'] += 1
            if word_comp['word_correct']:
                self.column_metrics[column]['correct_words'] += 1
            
            for char_comp in word_comp['char_comparisons']:
                self.column_metrics[column]['total_chars'] += 1
                if char_comp['is_correct']:
                    self.column_metrics[column]['correct_chars'] += 1
                else:
                    error_type = 'position_error' if char_comp['ocr_output']['char'] else 'substitution_error'
                    self.column_metrics[column]['error_types'][error_type] += 1

    def generate_report(self, column_results):
        """Generate comprehensive evaluation report"""
        report = {
            'overall_metrics': {},
            'column_metrics': {},
            'character_metrics': {},
            'position_metrics': {},
            'detailed_results': column_results
        }
        
        # Calculate overall metrics
        total_chars = sum(m['total'] for m in self.char_metrics.values())
        correct_chars = sum(m['correct'] for m in self.char_metrics.values())
        report['overall_metrics'] = {
            'total_characters': total_chars,
            'correct_characters': correct_chars,
            'character_accuracy': correct_chars / total_chars if total_chars > 0 else 0
        }
        
        # Calculate column-specific metrics
        for col, metrics in self.column_metrics.items():
            report['column_metrics'][col] = {
                'character_accuracy': metrics['correct_chars'] / metrics['total_chars'] if metrics['total_chars'] > 0 else 0,
                'word_accuracy': metrics['correct_words'] / metrics['total_words'] if metrics['total_words'] > 0 else 0,
                'error_distribution': dict(metrics['error_types'])
            }
        
        # Character-specific metrics
        report['character_metrics'] = pd.DataFrame([{
            'character': char,
            'total_occurrences': metrics['total'],
            'correct_recognitions': metrics['correct'],
            'accuracy': metrics['correct'] / metrics['total'] if metrics['total'] > 0 else 0,
            'position_errors': dict(metrics['position_errors']),
            'substitution_errors': dict(metrics['substitution_errors'])
        } for char, metrics in self.char_metrics.items()])
        
        # Position-specific metrics
        report['position_metrics'] = pd.DataFrame([{
            'position': pos,
            'total_occurrences': metrics['total'],
            'correct_recognitions': metrics['correct'],
            'accuracy': metrics['correct'] / metrics['total'] if metrics['total'] > 0 else 0,
            'common_errors': dict(metrics['errors'])
        } for pos, metrics in self.position_metrics.items()])
        
        return report

    def visualize_results(self, report, output_dir='./plots'):
        """Generate visualizations of the results"""
        import os
        os.makedirs(output_dir, exist_ok=True)
        
        # 1. Character Accuracy Heatmap
        plt.figure(figsize=(15, 8))
        char_accuracies = report['character_metrics'].pivot_table(
            values='accuracy',
            index='character'
        )
        sns.heatmap(char_accuracies, annot=True, cmap='YlGnBu', fmt='.2%')
        plt.title('Character Recognition Accuracy')
        plt.tight_layout()
        plt.savefig(f'{output_dir}/character_accuracy.png')
        plt.close()
        
        # 2. Position Accuracy Bar Plot
        plt.figure(figsize=(10, 6))
        pos_data = report['position_metrics']
        sns.barplot(x='position', y='accuracy', data=pos_data)
        plt.title('Accuracy by Character Position')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(f'{output_dir}/position_accuracy.png')
        plt.close()
        
        # 3. Column Performance Comparison
        plt.figure(figsize=(12, 6))
        col_accuracies = pd.DataFrame(report['column_metrics']).T
        col_accuracies[['character_accuracy', 'word_accuracy']].plot(kind='bar')
        plt.title('Accuracy by Column')
        plt.xlabel('Column')
        plt.ylabel('Accuracy')
        plt.legend(['Character Accuracy', 'Word Accuracy'])
        plt.tight_layout()
        plt.savefig(f'{output_dir}/column_accuracy.png')
        plt.close()

def evaluator():

    
    # Initialize and run evaluator
    evaluator = MultiColumnArabicEvaluator()
    results = evaluator.evaluate_dataframes( df_original_nid_1, df_predicted_nid)
    
    # Generate visualizations
    evaluator.visualize_results(results)
    
    return results

if __name__ == "__main__":
    results = evaluator()
    
    # Print summary results
    print("\nOverall Accuracy:", results['overall_metrics']['character_accuracy'])
    print("\nColumn-wise Accuracy:")
    for col, metrics in results['column_metrics'].items():
        print(f"\n{col}:")
        print(f"Character Accuracy: {metrics['character_accuracy']:.2%}")
        print(f"Word Accuracy: {metrics['word_accuracy']:.2%}")


Overall Accuracy: 0.7991718426501035

Column-wise Accuracy:

First Name:
Character Accuracy: 90.48%
Word Accuracy: 90.00%

Last Name:
Character Accuracy: 76.64%
Word Accuracy: 61.29%

Address:
Character Accuracy: 83.85%
Word Accuracy: 58.06%

District-Govrnate:
Character Accuracy: 77.01%
Word Accuracy: 69.70%


<Figure size 1200x600 with 0 Axes>

Other way for evaluation using sequence matcher technique

In [14]:
import pandas as pd
from difflib import SequenceMatcher

def calculate_accuracy(extracted_text, ground_truth_text):
    if pd.isna(extracted_text) and pd.isna(ground_truth_text):
        return 1.0
    elif pd.isna(extracted_text) or pd.isna(ground_truth_text):
        return 0.0
    return SequenceMatcher(None, str(extracted_text), str(ground_truth_text)).ratio()

def evaluate_ocr_results(extracted_df, ground_truth_df):
    if not extracted_df.columns.equals(ground_truth_df.columns):
        raise ValueError("Columns in extracted_df and ground_truth_df do not match")
    
    results = {}
    for column in extracted_df.columns:
        char_accuracies = []
        word_accuracies = []
        
        for ext, truth in zip(extracted_df[column], ground_truth_df[column]):
            char_accuracy = calculate_accuracy(ext, truth)
            
            # Word-level accuracy
            ext_words = str(ext).split() if not pd.isna(ext) else []
            truth_words = str(truth).split() if not pd.isna(truth) else []
            word_accuracy = calculate_accuracy(' '.join(ext_words), ' '.join(truth_words))
            
            char_accuracies.append(char_accuracy)
            word_accuracies.append(word_accuracy)
        
        results[column] = {
            'char_accuracy': sum(char_accuracies) / len(char_accuracies),
            'word_accuracy': sum(word_accuracies) / len(word_accuracies)
        }
    
    return results

def print_results(results):
    print("Evaluation Results:")
    for field, accuracies in results.items():
        print(f"\n{field}:")
        print(f"  Character-level accuracy: {accuracies['char_accuracy']:.2%}")
        print(f"  Word-level accuracy: {accuracies['word_accuracy']:.2%}")

In [15]:
results = evaluate_ocr_results(df_predicted_nid, df_original_nid_1)
print_results(results)

Evaluation Results:

First Name:
  Character-level accuracy: 98.89%
  Word-level accuracy: 98.89%

Last Name:
  Character-level accuracy: 85.35%
  Word-level accuracy: 85.35%

Address:
  Character-level accuracy: 81.19%
  Word-level accuracy: 81.19%

District-Govrnate:
  Character-level accuracy: 94.89%
  Word-level accuracy: 95.83%


Credit Card

In [16]:
inner_df=pd.merge(df_original_credit,df_predicted_credit, on="image_name", how="inner")
df_original_credit = inner_df[[col for col in inner_df.columns if '_x' in col] ].rename(columns=lambda x: x.replace('_x', ''))
df_predicted_credit = inner_df[[col for col in inner_df.columns if '_y' in col] ].rename(columns=lambda x: x.replace('_y', ''))

In [17]:
results = evaluate_ocr_results(df_predicted_credit, df_original_credit)
print_results(results)

Evaluation Results:

card_number:
  Character-level accuracy: 99.88%
  Word-level accuracy: 99.88%

card_holder:
  Character-level accuracy: 95.05%
  Word-level accuracy: 95.05%

expiry_date:
  Character-level accuracy: 98.62%
  Word-level accuracy: 98.62%


In [18]:
results = evaluate_ocr_results(df_predicted_credit, df_original_credit)
print_results(results)

Evaluation Results:

card_number:
  Character-level accuracy: 99.88%
  Word-level accuracy: 99.88%

card_holder:
  Character-level accuracy: 95.05%
  Word-level accuracy: 95.05%

expiry_date:
  Character-level accuracy: 98.62%
  Word-level accuracy: 98.62%
